# MPRA regression of background samples with K-fold cross validation

### Environment 
The next chunk contains the commands necessary to install the environment required to run this jupyter notebook
Skip this chunk if the installation was previously done

In [ ]:
%%bash
conda create --name tf_MPRA python=3.9.7
conda activate tf_MPRA
pip install tensorflow[and-cuda]
conda install -c anaconda ipykernel 
conda install -c anaconda pandas
conda install -c anaconda numpy
conda install -c anaconda scikit-learn 
conda install -c conda-forge matplotlib

# After installation if you are using VSCODE to run the notebook you have to close it and re-open

### Library imports


In [ ]:
import os 
import getopt
import sys

import random
import re

import numpy as np
import h5py
import pickle
import random
import copy
import pandas as pd
import math 

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Input, Masking, Lambda, concatenate, Bidirectional, Dense, Dropout, Flatten, Conv1D,BatchNormalization,  MaxPooling1D, Bidirectional, GRU, TimeDistributed
import tensorflow as tf
from tensorflow import keras



### Input ingestion

Here we define the methods to read and ingest data and we initialize the random seed.

Since we are processing the background the vocabulary is comprised of lower case nucleotides.

The upper cases (where the motif is), will be encoded as a zero-like vector

In [ ]:
np.random.seed(1337) # for reproducibility

# Lower case vocabulary
vocab = ["A", "C", "G", "T"]
indices = tf.range(len(vocab), dtype=tf.int64)
table_init = tf.lookup.KeyValueTensorInitializer(vocab, indices)
table = tf.lookup.StaticVocabularyTable(table_init, num_oov_buckets=1)

# These are the defaults of the data reader method 
# (each column in the ingested csv must be initialized with the right data type, otherwise the data ingestion fails )
defs = [0.] * 1 + [tf.constant([], dtype="string")] + [tf.constant([], dtype="string")]


def data_reader(file, batch_size=100, n_parse_threads=4):
    """Method for reading the data in an optimized way, can be used inside model.fit()
    
    Args:
        file (_type_): path to csv file
        batch_size (int, optional): _description_. Defaults to 100.
        n_parse_threads (int, optional): _description_. Defaults to 4.

    Returns:
        dataset.batch: batch dataset object 
    """
    dataset = tf.data.TextLineDataset(file).skip(1)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    return dataset.batch(batch_size).prefetch(1)

def preprocess(record):
    """Preprocessing method of a dataset object, one-hot-encodes the data

    Args:
        record (_type_): _description_

    Returns:
        X (2D np.array): one-hot-encoded input sequence
        Y (1D np.array): MPRA measurements

    """
    # Extract fields from passed batch
    fields = tf.io.decode_csv(record, record_defaults=defs)
    chars = tf.strings.bytes_split(fields[1])     # Extract sequences from 1st field
    chars_indices = table.lookup(chars)     # one-hot-encoding
    
    # Create a mask for out-of-vocabulary characters
    oov_mask = tf.equal(chars_indices, len(vocab))
    
    # Explicitly cast the value to int64
    chars_indices = tf.where(oov_mask, tf.constant(len(vocab), dtype=tf.int64), chars_indices)
    
    X = tf.one_hot(chars_indices, depth=len(vocab))
    Y = fields[0]
    return X, Y


### Randomization of motif sequences and data augmentation

This chunk defines a function that takes a dataframe with sequences and replaces upper case characters with random sequences N times

In [ ]:
def randomize_motifs_and_augment(df, num_augmentations=100):
    """This methods takes a dataframe containing a 'seq' column, randomizes the upper case characters and augments this sequences n times

    Args:
        df (pd.DataFrame): dataframe containing a 'seq' column
        num_augmentations (int, optional): Number of random sequences to obtaining for each input sequence. Defaults to 100.

    Returns:
        df (pd.DataFrame): df with an additional 'rnd_seq' column containing randomized upper case sequences, each sequence has N augmentation additional rows.
    """
    augmented_sequences = []

    for sequence in df['seq']:
        for _ in range(num_augmentations):
            random_sequence = ''.join(random.choice("ACGT") if char.isupper() else char for char in sequence)
            augmented_sequences.append(random_sequence)

    augmented_df = pd.DataFrame({'rnd_seq': augmented_sequences})
    df = df.loc[df.index.repeat(num_augmentations)].reset_index(drop=True)
    df = pd.concat([df, augmented_df], axis=1)
    return df

# Example data frame with DNA sequences

# test dataframe
data = {'seq': ["aaaACGTAGGCTA", "tttTTACGGTACACGT", "cccCGTACATACAGT"],
        'id' : ["1a", "2a", "3a"]}
df = pd.DataFrame(data)

# Call the randomize_motifs_and_augment function
result_df = randomize_motifs_and_augment(df, num_augmentations=3)

# Display the resulting data frame
result_df

### k-fold cross validation split
Here we take the initial csv file and we split it in 3 partitions k times

It is possible to randomize the sequences and augment, since the masking of the model motifs was a better choice
for understanding the background this strategy is here commented out and not used


In [ ]:
# CROSS VALIDATION (10 fold)
from sklearn.model_selection import train_test_split, KFold

whole_data = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/no_upper_LibA_wide_pivot_state3.csv"
out_data = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/"
# Split the data in two partitions
whole_data = pd.read_csv(whole_data)
k = 10
kf = KFold(n_splits = k, shuffle = True, random_state = 2008)

o=1
# For each fold we split again to get the third partition
for i in kf.split(whole_data):
    
    # Extract data from fold
    train = whole_data.iloc[i[0]]
    test =  whole_data.iloc[i[1]]
    
    # Split into train, validation and test
    valid, validation = train_test_split(train, test_size=0.111, random_state=42)
    
    # Randomize motifs
    #train = randomize_motifs_and_augment(train, num_augmentations=100)
    #train["rnd_seq"] = train['rnd_seq'].str.lower() 
    #test = randomize_motifs_and_augment(test, num_augmentations=100)
    #test["rnd_seq"] = test['rnd_seq'].str.lower() 
    #validation = randomize_motifs_and_augment(validation, num_augmentations=100)
    #validation["rnd_seq"] = validation['rnd_seq'].str.lower() 
    
    train[["State_3E", "seq", "CRS"]].to_csv(out_data+"background_CV"+str(o)+"_LibA_wide_pivot_state3_train.csv", index=False)
    test[["State_3E", "seq", "CRS"]].to_csv(out_data+"background_CV"+str(o)+"_LibA_wide_pivot_state3_test.csv", index=False)
    validation[["State_3E", "seq", "CRS"]].to_csv(out_data+"background_CV"+str(o)+"_LibA_wide_pivot_state3_validation.csv", index=False)
    o+=1
    

### Masking motifs

Since we do not want to backpropagate through the masked values we apply a mask on them

The mask is applied on values one-hot-encoded as a zero-like vector

Here we test the behaviour of the mask method to make sure it not masking 0 contained in every vector

In [ ]:
# Define the masking layer with mask_value=0
masking_layer = tf.keras.layers.Masking(mask_value=0.0)

# Example input tensor with shape (batch_size, sequence_length, vocab_size)
input_tensor = tf.constant([
    [[0, 0, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 0, 1, 0]],
    [[0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]]
], dtype=tf.float32)

# Apply the masking layer to the input tensor
masked_tensor = masking_layer(input_tensor)

print(masked_tensor)
masking_layer.compute_mask(input_tensor)

### Deep Learning model

Here we run the model which is based on this paper : 

https://doi.org/10.1101/2023.03.05.531189

I have added a Normalization layer parametrized with two parameters. Here we define the custom layer, the method to compute pearson correlation and a plotting function

In [ ]:
# Here we initialize the df where each fold test prediction will be appended to
# the list containing the correlations of each fold is also initialized
df_test_10folds  = pd.DataFrame(columns=['State_3E', "seq", "prediction"])
corr_list = []

# We define a custom normalization layer to then compile on the model
class CustomNormalization(Layer):
    """Custom normalization layer that normalizes the output of the neural network"""
    def __init__(self, **kwargs):
        super(CustomNormalization, self).__init__(**kwargs)
        
    def build(self, input_shape):
        # Add trainable variables for mean and standard deviation
        self.mean = self.add_weight("mean", shape=(1,), initializer="zeros", trainable=True)
        self.stddev = self.add_weight("stddev", shape=(1,), initializer="ones", trainable=True)
        super(CustomNormalization, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs):
        # Normalize the inputs using the learned mean and standard deviation
        return (inputs - self.mean) / (self.stddev + 1e-8)

# We define the method to compute the pearson correlation between prediction and ground truth
def pearson_correlation(x, y):
    """Computes Pearson Correlation between x and y
    Args:
        x (np.array): vector of predictions values
        y (np.array): vector of ground truth values

    Returns:
        (float): pearson correlation
    """
    n = len(x)
    
    # Calculate the mean of x and y
    mean_x = sum(x) / n
    mean_y = sum(y) / n
    
    # Calculate the numerator and denominators of the correlation coefficient
    numerator = sum((xi - mean_x) * (yi - mean_y) for xi, yi in zip(x, y))
    denominator_x = math.sqrt(sum((xi - mean_x) ** 2 for xi in x))
    denominator_y = math.sqrt(sum((yi - mean_y) ** 2 for yi in y))
    
    # Calculate the correlation coefficient
    correlation = numerator / (denominator_x * denominator_y)
    return correlation

import matplotlib.pyplot as plt
%matplotlib inline
# Define plotting function of loss
def create_plots(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    plt.clf()

#### Model training
Here we iterate through the folds and train the model

In [ ]:
df_test_10folds  = pd.DataFrame()
corr_list = []

for i in range(1,11):
    #Define inputs
    input_path_train = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/background_CV"+str(i)+"_LibA_wide_pivot_state3_train.csv"
    input_path_valid = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/background_CV"+str(i)+"_LibA_wide_pivot_state3_validation.csv"
    input_path_test = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/background_CV"+str(i)+"_LibA_wide_pivot_state3_test.csv"
   
    df_test = pd.read_csv(input_path_test)
    df_test["fold"] = str(i)
    corr_per_iteration = []
    # Get first item of the dataset to get the shape of the input data
    for element in data_reader(input_path_train):
        input_shape = element[0].shape
        
    for iteration in range(1,11):
        inputs = Input(shape=(input_shape[1],input_shape[2]), name="inputs")
        layer = Conv1D(250, kernel_size=7, strides=1, activation='relu', name="conv1")(inputs)  # 250 7 relu
        layer = Dropout(0.3)(layer)
        layer = BatchNormalization()(layer)
        layer = Conv1D(250, 8, strides=1, activation='softmax', name="conv2")(layer)  # 250 8 softmax
        layer = BatchNormalization()(layer)
        layer = MaxPooling1D(pool_size=2, strides=None, name="maxpool1")(layer)
        layer = Dropout(0.3)(layer)
        layer = Conv1D(250, 3, strides=1, activation='softmax', name="conv3")(layer)  # 250 3 softmax
        layer = BatchNormalization()(layer)
        layer = Dropout(0.3)(layer)
        layer = Conv1D(100, 2, strides=1, activation='softmax', name="conv4")(layer)  # 100 3 softmax
        layer = BatchNormalization()(layer)
        layer = MaxPooling1D(pool_size=1, strides=None, name="maxpool2")(layer)
        layer = Dropout(0.3)(layer)
        layer = Flatten()(layer)
        layer = Dense(300, activation='sigmoid')(layer)  # 300
        layer = Dropout(0.3)(layer)
        layer = Dense(200, activation='sigmoid')(layer)  # 300
        predictions = Dense(1, activation='linear')(layer)

        model = Model(inputs=inputs, outputs=predictions)
        model.compile(optimizer="adam",
                    loss="mean_squared_error",
                    metrics=["mse", "mae", "mape"],
                    )

        history=model.fit(data_reader(input_path_train, batch_size=100),
                                epochs=20,
                                validation_data=data_reader(input_path_valid,batch_size=100),
                                callbacks=None,
                                verbose=2)

        predicted = model.predict(data_reader(input_path_test,
                                                    batch_size=100))

        test_data = data_reader(input_path_test,batch_size=100)
        test_tensor = X = np.empty(shape=[0,1])
        for batch in test_data:
            test_tensor = np.append(test_tensor, batch[1])

        df_test["prediction_iteration_"+str(iteration)] = predicted
        
                
        corr_coefficient = pearson_correlation(predicted.flatten(), test_tensor)
        corr_per_iteration.append(corr_coefficient)
    
    df_test_10folds = pd.concat([df_test_10folds, df_test], ignore_index=True)    
        
    corr_ensemble = np.mean(corr_per_iteration)
    corr_list.append(corr_ensemble)

df_test_10folds.to_csv("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/LibA_wide_pivot_state3_test_predicted_cv10fold_motifs_only_ensemble.csv", index=False)

In [11]:
df_test_10folds

,State_3E,seq,CRS,fold,prediction_iteration_1,prediction_iteration_2,prediction_iteration_3,prediction_iteration_4,prediction_iteration_5,prediction_iteration_6,prediction_iteration_7,prediction_iteration_8,prediction_iteration_9,prediction_iteration_10
0,-0.005304,aggaccggatcaactaaacaactcaaacaagggctaatataaccca...,LibA.Seq7829,1,0.166644,0.008744,0.079746,0.205307,0.077799,0.092063,0.154507,0.082586,0.092175,0.120026
1,0.236881,aggaccggatcaactaaacactagtcatacttaaaaattgcaagga...,LibA.Seq271,1,0.009147,-0.044254,0.058593,-0.053590,0.069522,0.017253,-0.004010,0.030977,0.016632,0.065043
2,-0.086491,aggaccggatcaactaaacaggttctgacgtatgctcctctatgga...,LibA.Seq4548,1,0.019830,-0.027810,0.022879,0.063743,0.031034,0.008034,0.017735,0.037538,0.060172,0.028070
3,-0.087684,aggaccggatcaactaaacccgagcctgcctagccctagcttctct...,LibA.Seq4582,1,0.005316,0.004229,0.022408,0.027580,0.063660,0.021920,0.009619,0.037809,0.055539,0.026170
4,0.371111,aggaccggatcaactaaacggagcagagttagtgtcaggtcaaaaa...,LibA.Seq2863,1,0.177948,0.104138,0.251513,0.305862,0.222175,0.221340,0.207982,0.224182,0.204155,0.198397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8487,-0.076574,aggaccggatcaacttttggtcggttgacggtcgccttgattattc...,LibA.Seq4154,10,0.025850,-0.119319,0.059446,0.065505,-0.003330,0.032118,0.084711,0.060746,0.052425,0.102655
8488,0.369648,aggaccggatcaactttttcagtgaaagatcaccgcgggatctcac...,LibA.Seq8531,10,0.471889,0.022261,0.221582,0.396499,0.270562,0.340061,0.331227,0.339184,0.368296,0.206447
8489,-0.098525,aggaccggatcaacttttttagtaaaactcttaaacagtgattaca...,LibA.Seq6744,10,0.213882,-0.037019,0.103241,0.078904,0.061012,0.094831,0.164944,0.172287,0.074359,0.275276
8490,0.164442,aggaccggatcaacttttttatctggttatcattctagtctagtgc...,LibA.Seq1298,10,0.154465,-0.016604,0.095095,0.145902,0.082548,0.228232,0.094881,0.129145,0.094053,0.110160


In [ ]:
create_plots(history)